In [8]:
import io

import requests
from bs4 import BeautifulSoup
from termcolor import colored


def exact_matcher(lhs, rhs):
    return lhs == rhs


def float_matcher(lhs, rhs):
    return abs(float(lhs) - float(rhs)) <= 10**-7


class Algospot:
    def __init__(self, problem_id, matcher=None):
        self._problem_id = problem_id
        self._matcher = exact_matcher if matcher is None else matcher
        
        self._page = None
        
    def overwrite(self):
        return self._input(), self._print()
    
    def _input(self):
        tag = self.page.find('section', class_='problem_sample_input').find('pre')
        return io.StringIO(tag.text).readline
    
    def _print(self):
        tag = self.page.find('section', class_='problem_sample_output').find('pre')
        correct_answers = iter(io.StringIO(tag.text).readlines())

        def grading(answer):
            color = 'red'
            answer = str(answer).strip()
            correct_answer = correct_answers.__next__().strip()
            if self._matcher(answer, correct_answer):
                color = 'green'
            __builtin__.print(colored(answer, color))
            
        return grading
    
    @property
    def problem_url(self):
        return 'https://algospot.com/judge/problem/read/{}'.format(self._problem_id)
    
    @property
    def page(self):
        if self._page is None:
            html = requests.get(self.problem_url)
            self._page = BeautifulSoup(html.text, "html5lib")
            
        return self._page
